In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Openpose model download
!git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose.git /content/drive/MyDrive/openpose
%cd /content/drive/MyDrive/openpose
%cd /content/drive/MyDrive/openpose
!bash /content/drive/MyDrive/openpose/models/getModels.sh

fatal: destination path '/content/drive/MyDrive/openpose' already exists and is not an empty directory.


In [ ]:
import cv2
import numpy as np
import os
import csv
from tqdm import tqdm

# OpenPose 모델 파일 경로
protoFile_COCO = "/content/drive/MyDrive/openpose/models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile_COCO = "/content/drive/MyDrive/openpose/pose/coco/pose_iter_440000.caffemodel"

# 상반신 키포인트 정의
BODY_PARTS_UPPER = {
    0: "Nose", 1: "Neck",
    2: "RShoulder", 3: "RElbow", 4: "RWrist",
    5: "LShoulder", 6: "LElbow", 7: "LWrist",
    8: "RHip", 11: "LHip",
    14: "REye", 15: "LEye", 16: "REar", 17: "LEar"
}

POSE_PAIRS_UPPER = [
    [0, 1], [0, 14], [0, 15],
    [1, 2], [1, 5],
    [2, 3], [3, 4],
    [5, 6], [6, 7],
    [14, 16], [15, 17],
    [2, 8], [5, 11]
]

# 네트워크 로드
net_openpose = cv2.dnn.readNetFromCaffe(protoFile_COCO, weightsFile_COCO)
net_openpose.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net_openpose.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# 경로 설정
video_dir = "/content/drive/MyDrive/good_clips"
output_dir = "/content/drive/MyDrive/good_clip"
os.makedirs(output_dir, exist_ok=True)

# 최종 CSV 저장 경로
combined_csv_path = os.path.join(output_dir, "all_videos_keypoints.csv")

# 키포인트 추출 함수
def output_keypoints(frame, threshold=0.1):
    global points

    image_height, image_width = 368, 368
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    net_openpose.setInput(input_blob)
    out = net_openpose.forward()

    frame_height, frame_width = frame.shape[:2]
    out_height, out_width = out.shape[2], out.shape[3]

    points = []
    for i in BODY_PARTS_UPPER.keys():
        prob_map = out[0, i, :, :]
        _, prob, _, point = cv2.minMaxLoc(prob_map)

        x = int((frame_width * point[0]) / out_width)
        y = int((frame_height * point[1]) / out_height)

        if prob > threshold:
            points.append((x, y))
        else:
            points.append(None)

# 키포인트 연결선 그리기 함수 (이미지 저장용)
def output_keypoints_with_lines(frame):
    for pair in POSE_PAIRS_UPPER:
        part_a, part_b = pair
        if part_a >= len(points) or part_b >= len(points):
            continue
        if points[part_a] is None or points[part_b] is None:
            continue
        cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 2)
    # 점도 같이 찍어줌
    for i, point in enumerate(points):
        if point is not None:
            cv2.circle(frame, point, 5, (0, 255, 255), -1)
            cv2.putText(frame, BODY_PARTS_UPPER.get(i, str(i)), (point[0] + 5, point[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# CSV 초기화
with open(combined_csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # 헤더 구성
    header = ['filename', 'frame']
    for part in BODY_PARTS_UPPER.values():
        header.extend([f'{part}_x', f'{part}_y'])
    csv_writer.writerow(header)

    # 폴더 내 모든 영상 순회
    for video_file in os.listdir(video_dir):
        if not video_file.endswith(('.mp4', '.avi', '.mov')):
            continue

        video_path = os.path.join(video_dir, video_file)
        capture = cv2.VideoCapture(video_path)

        fps_video = int(capture.get(cv2.CAP_PROP_FPS))
        total_frame = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

        # 1초에 1프레임씩 처리
        fps_target = 1
        frame_interval = fps_video // fps_target

        # 영상별 저장 폴더 생성 (이미지 저장용)
        video_output_dir = os.path.join(output_dir, video_file.split('.')[0])
        os.makedirs(video_output_dir, exist_ok=True)

        print(f"▶ Processing {video_file}...")

        for now_frame in tqdm(range(0, total_frame, frame_interval), desc=f"{video_file} 진행중"):
            capture.set(cv2.CAP_PROP_POS_FRAMES, now_frame)
            ret, frame = capture.read()

            if not ret:
                print(f"{video_file} 프레임 {now_frame} 읽기 실패. 넘어감.")
                break

            frame_openpose = frame.copy()

            output_keypoints(frame_openpose, threshold=0.1)
            output_keypoints_with_lines(frame_openpose)

            # 키포인트 좌표 CSV에 저장
            row = [video_file, now_frame]
            for point in points:
                if point is None:
                    row.extend([None, None])
                else:
                    row.extend([point[0], point[1]])
            csv_writer.writerow(row)

            # 프레임별 이미지 저장
            frame_img_path = os.path.join(video_output_dir, f"{now_frame:04d}.jpg")
            cv2.imwrite(frame_img_path, frame_openpose)

        capture.release()

print(f"✅ 모든 작업 완료! CSV 저장: {combined_csv_path}")


▶ Processing A6Dkt7zyImk_043.mp4...


A6Dkt7zyImk_043.mp4 진행중: 100%|██████████| 4/4 [00:26<00:00,  6.71s/it]


▶ Processing 3UqkgRGPqsk_068.mp4...


3UqkgRGPqsk_068.mp4 진행중: 100%|██████████| 5/5 [00:37<00:00,  7.45s/it]


▶ Processing 3E7hkPZ-HTk_084.mp4...


...

TS6lFDVR-3g_125.mp4 프레임 0 읽기 실패. 넘어감.


▶ Processing A6Dkt7zyImk_000.mp4...


A6Dkt7zyImk_000.mp4 진행중: 100%|██████████| 6/6 [00:41<00:00,  6.85s/it]


▶ Processing A6Dkt7zyImk_001.mp4...


A6Dkt7zyImk_001.mp4 진행중:   0%|          | 0/9 [00:00<?, ?it/s]